In [1]:

from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM, T5Tokenizer, T5ForConditionalGeneration, LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
import torch
from langchain.llms import HuggingFacePipeline

tokenizer = LlamaTokenizer.from_pretrained("chavinlo/alpaca-native")

base_model = LlamaForCausalLM.from_pretrained(
    "chavinlo/alpaca-native",
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map='auto',
)

pipe = pipeline(
    "text-generation",
    model=base_model, 
    tokenizer=tokenizer, 
    max_length=256,
)

local_llm = HuggingFacePipeline(pipeline=pipe)

c:\Users\Aviv9\miniconda3\envs\LLMBots\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: C:\Users\Aviv9\miniconda3\envs\LLMBots\bin\cudart64_110.dll
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary c:\Users\Aviv9\miniconda3\envs\LLMBots\lib\site-packages\bitsandbytes\libbitsandbytes_cuda117.dll...


c:\Users\Aviv9\miniconda3\envs\LLMBots\lib\site-packages\bitsandbytes\cuda_setup\main.py:141: UserWarning: C:\Users\Aviv9\miniconda3\envs\LLMBots did not contain cudart64_110.dll as expected! Searching further paths...
  warn(msg)
c:\Users\Aviv9\miniconda3\envs\LLMBots\lib\site-packages\bitsandbytes\cuda_setup\main.py:141: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('C:/Program Files (x86)/Razer/ChromaBroadcast/bin'), WindowsPath('C:/WINDOWS/system32/config/systemprofile/AppData/Local/Microsoft/WindowsApps'), WindowsPath('C:/Program Files (x86)/Razer Chroma SDK/bin'), WindowsPath('C:/Program Files/Razer Chroma SDK/bin'), WindowsPath('C:/ProgramData/DockerDesktop/version-bin'), WindowsPath('C:/Users/Aviv9/miniconda3/envs/LLMBots/Library/mingw-w64/bin'), WindowsPath('C:/Program Files/Razer/ChromaBroadcast/bin'), WindowsPath('C:/Program Files/JetBrains/DataSpell 2021.3.2/bin')}
  warn(msg)
Loading checkpoint shards: 100%|

In [2]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

template = """Question: {question}

Answer: """

prompt = PromptTemplate(template=template, input_variables=["question"])

In [3]:
llm_chain = LLMChain(prompt=prompt, llm=local_llm)
question = "What is the capital of England?"

print(llm_chain.run(question))

c:\Users\Aviv9\miniconda3\envs\LLMBots\lib\site-packages\transformers\generation\utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


 The capital of England is London.


In [10]:
general_directions = """
Your decisions must always be made independently without seeking user assistance. Play to your strengths as an LLM and pursue simple strategies with no legal complications.


CONSTRAINTS:
1. No user assistance.
2. Cannot run Python code that requires user input.


ACTIONS:

1. "READ_FILE": read the current state of a file. The schema for the action is:

READ_FILE: <PATH>

2. "WRITE_FILE": write a block of text to a file. The schema for the action is:

WRITE_FILE: <PATH>
```
<TEXT>
```

3. "RUN_PYTHON": run a Python file. The schema for the action is:

RUN_PYTHON: <PATH>

4. "SEARCH_ONLINE": search online and get back a list of URLs relevant to the query. The schema for the action is:

SEARCH_ONLINE: <QUERY>

5. EXTRACT_INFO: extract specific information from a webpage. The schema for the action is:

EXTRACT_INFO: <URL>, <a brief instruction to GPT for information to extract>

6. "SHUTDOWN": shut down the program. The schema for the action is:

SHUTDOWN


RESOURCES:
1. File contents after reading file.
2. Online search results returning URLs.
3. Output of running a Python file.


PERFORMANCE EVALUATION:
1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities. 
2. Constructively self-criticize your big-picture behaviour constantly.
3. Reflect on past decisions and strategies to refine your approach.
4. Every action has a cost, so be smart and efficent. Aim to complete tasks in the least number of steps.


Write only one action. The action must one of the actions specified above and must be written according to the schema specified above.

After the action, also write the following metadata JSON object, which must be parsable by Python's json.loads():


If you want to run an action that is not in the above list of actions, send the SHUTDOWN action instead and explain which action you wanted to run in the metadata JSON object.
So, write one action and one metadata JSON object, nothing else.
Goal: {goal}
AI:
"""

In [11]:
robo_agent = LLMChain(prompt=PromptTemplate(template=general_directions, input_variables=["goal"]), llm=HuggingFacePipeline(pipeline=pipe))

In [12]:
robo_agent.run("I want you to increase my profile searches in linkedin")

c:\Users\Aviv9\.virtualenvs\Robo-GPT-QaSuWWH8\lib\site-packages\transformers\generation\utils.py:1245: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Input length of input_ids is 572, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
c:\Users\Aviv9\.virtualenvs\Robo-GPT-QaSuWWH8\lib\site-packages\transformers\generation\utils.py:1428: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling f

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)